<a href="https://colab.research.google.com/github/good0neighbour/ESTCampAI/blob/main/TextMining/ex02_%EA%B0%90%EC%A0%95%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensim
# gensim 설치 후 세션 다시 시작.

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd /content/drive/MyDrive/ESTCampAI/TextMining
# 세션 다시 시작 후에는 경로가 초기화되어 다시 실행해주어야 한다.

/content/drive/MyDrive/ESTCampAI/TextMining


### 네이버 쇼핑 리뷰 데이터 기반으로 임베딩 맛보기

In [4]:
import pandas as pd
data = pd.read_csv('./data/naver_shopping.txt', delimiter='\t', header = None)

In [5]:
data.head(3)

,0,1
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...


In [6]:
# 컬럼명 업데이트
data.columns = ['평점','리뷰']

In [7]:
data.head()

,평점,리뷰
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


In [8]:
# 리뷰(document)만 인덱싱
text = data['리뷰']

### 1. 정제
- 1.1 결측치 확인
- 1.2 특수문자 제거

In [9]:
data.info()
# 결측치 없음 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   평점      200000 non-null  int64 
 1   리뷰      200000 non-null  object
dtypes: int64(1), object(1)
memory usage: 3.1+ MB


- 1.2 특수문자 제거 -> re 활용

In [10]:
import re

# r"" : 원시(raw)문자열, \를 하나만 사용 가능하게끔 만듬!
pattern = r'[^a-zA-Z0-9가-힣\s\.\?\!]'

# 알파벳, 숫자, 한글문자, 공백을 제외한 패턴
# 리뷰가 길어지는 경우, 문장 구분을 위해 공백(\s), .?! 보존
# "\s"는 공백 문자를 의미하며, 여기에는 줄바꿈(\n)도 포함됨
# ^ : not

In [11]:
from tqdm import tqdm

new_doc = []
for doc in tqdm(text):
  cleaned_doc = re.sub(pattern, '', doc)
  new_doc.append(cleaned_doc)

100%|██████████| 200000/200000 [00:00<00:00, 232359.44it/s]


In [12]:
# 정제 된 텍스트 데이터 컬럼으로 추가
data['cleaned_doc'] = new_doc

In [13]:
data.head(3)

,평점,리뷰,cleaned_doc
0,5,배공빠르고 굿,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...


```
!pip install git+https://github.com/ssut/py-hanspell.git
```

```from hanspell import spell_checker
import time
from tqdm import tqdm

right_spell = []

for doc in tqdm(new_doc):  # new_doc: 문장 리스트
    try:
        corrected = spell_checker.check(doc)
        right_spell.append(corrected.checked)
        time.sleep(0.5)  # 차단 방지
    except Exception as e:
        right_spell.append(doc)  # 오류 시 원문 저장
```

### 2. 형태소 분리
- 2.1 형태소 분리, 어간 추출, 정규화, 원하는 품사 추출, 불용어 제거, 띄어쓰기 교정

In [14]:
!pip install kiwipiepy

In [15]:
!pip install konlpy

In [16]:
# 형태소 분석기 초기화
from konlpy.tag import Okt
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords

okt = Okt()
kiwi = Kiwi()
stopwords = Stopwords()

In [17]:
stopwords_list = [word for word, tag in stopwords.stopwords]

In [18]:
# 사용자 함수 정의하기
def pos_tagging(text):
  # 1. 띄어쓰기 교정
  text = kiwi.space(text)

  # 2. 품사태깅(형태소 분리)
  pos_words = okt.pos(
      text,
      stem = True, # 어간추출
      norm = True # 덩규화(표준화)
  )

  # 3. 원하는 품사 추출(명사, 형용사, 동사)
  tagged_list = []

  for word, tag in pos_words:
    if tag in ['Noun', 'Adjective', 'Verb']: # 명사, 형용사, 동사인가?
      if word not in stopwords_list: # 불용어 사전에 해당되지 않는가?
        tagged_list.append(word)

  return tagged_list

In [19]:
# 함수 적용
#tagged_doc = []

#for text in tqdm(data['cleaned_doc']): # 모든 정제리뷰를 순회
#  tagged_doc.append(pos_tagging(text)) # 형태소 분리 및 전처리된 text를 tagged_doc에 추가

In [20]:
import pickle
import pandas as pd
from tqdm import tqdm

# DataFrame에 list 자료형은 csv로 저장하는 경우에 데이터타입이 문자열로 변경되어버리기 때문에 pkl등 다른 확장자를 이용하는 것이 좋다.
data = pd.read_pickle('data/naver_shopping(정제 및 토큰화 완료).pkl')
data.head()

,평점,리뷰,cleaned_doc,tagged_doc
0,5,배공빠르고 굿,배공빠르고 굿,"[배공, 빠르다, 굿]"
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,"[택배, 엉망, 용, 저희, 집, 밑, 층, 놔두다, 가다]"
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,"[아주, 좋다, 바지, 정말, 좋다, 개, 구매, 하다, 가격, 대박, 이다, 바느..."
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,"[선물, 용, 받다, 전달, 하다, 하다, 상품, 이다, 머, 컵, 오다, 당황, ..."
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요,"[민트, 색상, 예쁘다, 옆, 손잡이, 거, 용, 도로, 사용, 되다]"


### Word2Vec
- 단어 임베딩 방법론(모델) 중 하나
> 두 가지 방법<br>
> CBOW : 주변에 있는 단어를 입력으로 중간에 있는 단어를 학습/예측하는 방법<br>
> Skip-gram : 중간에 있는 단어를 입역으로 주변 단어 학습/예측하는 방법

In [21]:
# word2vec 모델을 활용한 임베딩 알아보기
import gensim.downloader as api
# gensim : 복잡한 설치 없이 간단한 api 호출로 사전 학습된 워드 임베딩 모델 제공

In [22]:
# 사용가능한 여러 모델 리스트업
for model_name, model_data in sorted(api.info()['models'].items()):
    num_records = model_data.get('num_records', -1)
    description = model_data['description'][:40] + '...'

    print(f"{model_name} ({num_records} records): {description}")

__testing_word2vec-matrix-synopsis (-1 records): [THIS IS ONLY FOR TESTING] Word vecrors ...
conceptnet-numberbatch-17-06-300 (1917247 records): ConceptNet Numberbatch consists of state...
fasttext-wiki-news-subwords-300 (999999 records): 1 million word vectors trained on Wikipe...
glove-twitter-100 (1193514 records): Pre-trained vectors based on  2B tweets,...
glove-twitter-200 (1193514 records): Pre-trained vectors based on 2B tweets, ...
glove-twitter-25 (1193514 records): Pre-trained vectors based on 2B tweets, ...
glove-twitter-50 (1193514 records): Pre-trained vectors based on 2B tweets, ...
glove-wiki-gigaword-100 (400000 records): Pre-trained vectors based on Wikipedia 2...
glove-wiki-gigaword-200 (400000 records): Pre-trained vectors based on Wikipedia 2...
glove-wiki-gigaword-300 (400000 records): Pre-trained vectors based on Wikipedia 2...
glove-wiki-gigaword-50 (400000 records): Pre-trained vectors based on Wikipedia 2...
word2vec-google-news-300 (3000000 records): Pre-trai

- Records : 레코드의 수, 즉 데이터의 개수를 의미(각 "레코드"는 임베딩된 단어 하나를 의미)

| 모델 이름                           | 레코드 수      | 설명                                                                                                  |
|-------------------------------------|----------------|-------------------------------------------------------------------------------------------------------|
| conceptnet-numberbatch-17-06-300    | 1,917,247      | ConceptNet Numberbatch는 개념 간의 관계를 학습한 모델.                                                 |
|                                     |                | 상식을 이해하고 개념 간 관계를 연결시키기 위해 개발됨.                                                 |
|                                     |                | 주로 의미 네트워크 연구에 사용됨. "고양이"와 "동물" 간의 관계를 학습.                                  |
| fasttext-wiki-news-subwords-300     | 999,999        | fastText는 단어뿐만 아니라 서브워드(단어의 작은 부분)를 학습.                                          |
|                                     |                | "unbelievable"을 "un", "believe", "able"로 나누어 학습.                                                 |
|                                     |                | Wikipedia와 뉴스 기사 데이터를 기반으로 학습됨.                                                       |
| glove-twitter-100                   | 1,193,514      | GloVe는 Stanford에서 개발된 모델로, 단어 간의 공간적 관계를 벡터로 표현.                               |
|                                     |                | 트위터 데이터(2억 개 트윗)로 학습되어 소셜 미디어 자연어 처리 작업에 적합.                             |
| word2vec-google-news-300            | 3,000,000      | Word2Vec은 구글 뉴스 데이터로 학습된 모델.                                                             |
|                                     |                | 뉴스 기사의 단어 간 유사성 및 의미적 관계를 잘 포착.                                                   |
|                                     |                | 뉴스 데이터 기반이기 때문에 자연어 처리에 많이 사용됨.                                                 |
| word2vec-ruscorpora-300             | 184,973        | Word2Vec ruscorpora는 러시아어 텍스트 기반의 모델로,                                                   |
|                                     |                | 러시아어 자연어 처리 작업에 사용됨.                                                                   |

In [23]:
# gensim에서는 여러개의 사전학습 모델이 제공됨
# 이 중에서 word2vec-google-news-300 모델 로드하기
model = api.load('word2vec-google-news-300')
# 구글 뉴스 데이터로 훈련된 Word2Vec 모델
# 3백만 개의 단어 벡터를 포함하고 있으며, 각 단어는 300차원 벡터로 표현

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [24]:
# 'cat' 단어의 임베딩 벡터를 가져옴
cat_vector = model['cat']
len(cat_vector)

300

In [25]:
# 'cat'이라는 단어에 대응하는 300차원 벡터가 출력
cat_vector.shape

(300,)

In [26]:
# 유사도를 계산하여, 주어진 단어와 벡터 공간성에서 가장 가까운 단어를 나열
# 기본 코사인 유사도 점수 (-1 ~ 1 사이의 값, 벡터가 동일한 방향일 수록 값은 1에 가까워지고, 벡터가 반대방향일수록 -1에 가까워진다)
model.most_similar('cat')

[('cats', 0.8099379539489746),
 ('dog', 0.760945737361908),
 ('kitten', 0.7464985251426697),
 ('feline', 0.7326234579086304),
 ('beagle', 0.7150582671165466),
 ('puppy', 0.7075453400611877),
 ('pup', 0.6934291124343872),
 ('pet', 0.6891531348228455),
 ('felines', 0.6755931973457336),
 ('chihuahua', 0.6709762215614319)]

-1 이상 1이하의 값을 가지며 값이 1에 가까울수록 유사도가 높다고 판단
- 두 벡터가 가리키는 방향이 얼마나 유사한가를 의미
<img src = "https://wikidocs.net/images/page/24603/코사인유사도.PNG"/>

> **결과 해석:**  
> - 두 벡터가 완전히 동일한 방향이면(각도 $(\theta = 0$)) 유사도는 1  
> - 두 벡터가 서로 직교(각도 $(90^\circ$))하면 유사도는 0  
> - 두 벡터가 반대 방향이면(각도 $(\theta = 180^\circ$)) 유사도는 -1  
> - **문서 유사도 계산** 시 보통 음수 값은 잘 안 나오므로(벡터가 음수값을 가질 때 제외), 0에 가까울수록 유사도가 낮고 1에 가까울수록 유사도가 높다고 해석함  